In [1]:
# Create a One-To-Many Sqlite3 table

import sqlite3

# Connect to sqlite database
conn = sqlite3.connect('nominations.db')

# Query the table for schema info
query_table = "pragma table_info(nominations);"
schema = conn.execute(query_table).fetchall()

# Query the table for the first 10 instances
query_first_ten = "Select * from nominations Limit 10;"
first_ten = conn.execute(query_first_ten).fetchall()

# Print out the elements row by row
i = 1
for row in first_ten:
    print("Row: ", i ," ",row,'\n')
    i +=1

for row in schema:
    print("Schema: ", row)


Row:  1   (1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, 1) 

Row:  2   (2, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', 0, 1) 

Row:  3   (3, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', 0, 1) 

Row:  4   (4, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech", 'King George VI', 1, 1) 

Row:  5   (5, 'Actor -- Leading Role', 'James Franco', '127 Hours', 'Aron Ralston', 0, 1) 

Row:  6   (6, 'Actor -- Supporting Role', 'Christian Bale', 'The Fighter', 'Dicky Eklund', 1, 1) 

Row:  7   (7, 'Actor -- Supporting Role', 'John Hawkes', "Winter's Bone", 'Teardrop', 0, 1) 

Row:  8   (8, 'Actor -- Supporting Role', 'Jeremy Renner', 'The Town', 'James Coughlin', 0, 1) 

Row:  9   (9, 'Actor -- Supporting Role', 'Mark Ruffalo', 'The Kids Are All Right', 'Paul', 0, 1) 

Row:  10   (10, 'Actor -- Supporting Role', 'Geoffrey Rush', "The King's Speech", 'Lionel Logue', 0, 1) 

Schema:  (0, 'id', 'inte

In [2]:
# Create a one to many relationship by creating a new table that has
# the host of each years ceremony that the nominations table can 
# reference.

# Create new table
create_table = '''CREATE TABLE ceremonies (
 id integer PRIMARY KEY,
 Year integer,
 Host text
);'''
conn.execute(create_table)

# Populate new table with data
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]

insert_query = "Insert Into ceremonies (Year, Host) Values (?,?);"
conn.executemany(insert_query, years_hosts)

# Check to see if all data was entered properly
data_query = "Select * from ceremonies Limit 10;"
print(conn.execute(data_query).fetchall())

schema_query = "pragma table_info(ceremonies);"
print(conn.execute(schema_query).fetchall())

OperationalError: table ceremonies already exists

In [17]:
# Turn on Foreign Key constraints, ensuring that if new data is 
# added to either table, the foreign key will map properly to the 
# corresponding table.

pragma_fk_on = "Pragma foreign_keys = On;"

conn.execute(pragma_fk_on)

In [16]:
# We need to change elements in the nominations table allowing us 
# to connect the two tables togeter, with SQLITE however, that 
# means we are required to create a new table with the elements 
# we want. Then we can delete the old table.

create_new_table = '''Create Table nominations_two (
id integer Primary Key,
category text,
nominee text,
movie text,
character text,
won integer,
ceremony_id integer,
FOREIGN KEY(ceremony_id) REFERENCES ceremonies(id)
);'''

conn.execute(create_new_table)

In [15]:
# Gather data to insert into new table

joined_noms_query = '''Select nominations.category, 
nominations.nominee, nominations.movie, nominations.character, 
nominations.won, ceremonies.id
From nominations
Inner Join ceremonies On
nominations.year == ceremonies.year;
'''
joined_nominations = conn.execute(joined_noms_query).fetchall()

# Create a placeholder statement and insert data from nominations
# into nominations_two table
place_holder_statement = '''Insert Into nominations_two (Category,
Nominee,Movie,Character,Won,ceremony_id) Values (?,?,?,?,?,?);
'''
conn.executemany(place_holder_statement,joined_nominations)

# Ensure data was inserted correctly
query_5 = "Select * from nominations_two Limit 5;"
print(conn.execute(query_5).fetchall())

OperationalError: no such table: nominations

In [14]:
# Delete the old nominations table we no longer will be using
drop_noms1 = "Drop Table nominations;"
conn.execute(drop_noms1)

# Alter the nominations_two folder and change name to nominations
# now that the old nominations has been deleted
alter_noms2 = "Alter Table nominations_two Rename To nominations;"
conn.execute(alter_noms2)

OperationalError: no such table: nominations_two

In [12]:
# Create a Many-To-Many Sqlite3 tables

# Create all the tables and the join tables before populating them

create_movies_table = '''Create Table movies (
id integer Primary Key,
movie text
)'''
create_actors_table = '''Create Table actors (
id integer Primary Key,
actor text
)'''
create_join_table = '''Create Table movies_actors (
id integer Primary Key,
movie_id integer,
actor_id integer,
Foreign Key(movie_id) References movies(id),
Foreign Key(actor_id) References actors(id)
)'''

conn.execute(create_movies_table)
conn.execute(create_actors_table)
conn.execute(create_join_table)

OperationalError: table movies already exists

In [8]:

# Populate movies table
movies_query = "Select nominations.movie from nominations;"
movie_names = conn.execute(movies_query).fetchall()

movie_place_holder = "Insert Into movies (movie) Values (?);"
conn.executemany(movie_place_holder, movie_names)

# Populate actors table
actors_query = "Select nominations.nominee from nominations;"
actor_names = conn.execute(actors_query).fetchall()

actor_place_holder = "Insert Into actors (actor) Values (?);"
conn.executemany(actor_place_holder, actor_names)

# Populate join table
movies_actors_query = "Select movie,nominee from nominations;"
movie_actor_names = conn.execute(movies_actors_query).fetchall()

movie_actor_place_holder = '''Insert Into movies_actors 
(movie_id, actor_id) Values (
(Select id from movies Where movie == ?),
(Select id from actors Where actor == ?)
);'''
conn.executemany(movie_actor_place_holder, movie_actor_names)



In [11]:
# Now, let's check the data to make sure our join tables have worked!
check_data_query = "Select * from movies_actors Limit 10;"
check_data = conn.execute(check_data_query).fetchall()
print(check_data)

[(1, 1, 1), (2, 2, 2), (3, 3, 3), (4, 4, 4), (5, 5, 5), (6, 6, 6), (7, 7, 7), (8, 8, 8), (9, 9, 9), (10, 4, 10)]
